# OpenFoam with F3DASM: flow around a cylinder

This tutorial is directly inspired from the flow around a cylinder case discribed [here](https://www.openfoam.com/documentation/tutorial-guide/2-incompressible-flow/2.2-flow-around-a-cylinder) and whose geoemetry is reported below.

![tutorial geometry](https://www.openfoam.com/documentation/tutorialguide/img/tutorial137x.png)

In this tutorial, the input velocity, output pressure and mesh size are automatically drawn from a design of experiemt generated with F3DASM. The reulting error with respect to the analytical solution is extracted and stored.

> **_NOTE:_**  This tutorial was tested with openfoam10. Installation instructions can be found [here](https://openfoam.org/version/10/). However, it should be compatible with the both main OpenFoam releases, from OpenCFD Ltd. and OpenFoam Foundation. Please report any issues using the Github issue interface.


## Imports

In [1]:
%load_ext autoreload
import f3dasm
from f3dasm_simulate import openFoam
from PyFoam import FoamInformation
from pathlib import Path
from operator import itemgetter

2023-06-27 00:17:40,353 - Imported f3dasm (version: 1.1.0)
2023-06-27 00:17:40.856193: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-27 00:17:40.857490: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 00:17:40.883627: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 00:17:40.884475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 00:17:41.295468: W tensorflow/compiler/tf2t

Be sure to load openFOam to path before running the notebook, for instance by running `source /opt/openfoam10/etc/bashrc`

In [2]:
# The following statement should return openFoam version
print(f"openFoam version: {FoamInformation.foamVersion()}")

openFoam version: 10


## Create the Design of Experiment (DoE)

In [3]:
%autoreload 2
# Create the input parameters
mesh_size = f3dasm.DiscreteParameter(
    name="mesh_factor", lower_bound=1, upper_bound=10
)
input_velocity = f3dasm.ContinuousParameter(
    name="input_velocity", lower_bound=0.1, upper_bound=10.0
)
output_pressure = f3dasm.ContinuousParameter(
    name="output_pressure", lower_bound=0.0, upper_bound=1.0
)

# Create the output quantities
avg_error = f3dasm.ContinuousParameter(name="avg_error")
max_error = f3dasm.ContinuousParameter(name="max_error")

# Define the design space
design = f3dasm.DesignSpace(
    input_space=[mesh_size, input_velocity, output_pressure],
    output_space=[avg_error, max_error],
)

# Create the sampler object
sampler = f3dasm.sampling.RandomUniform(design=design, seed=42)

# Sample the design space according to the DoE
data: f3dasm.ExperimentData = sampler.get_samples(numsamples=10)

job_queue = f3dasm.experiment.JobQueue.from_experimentdata(
    filename="my_jobs", experimentdata=data
)

job_queue.write_new_jobfile()

## Run simulators parametrized by the DoE

In [4]:
%autoreload 2

def error_calculations(self):
    """Helper functions to process the results."""
    error_field = self.solution_dir.getDictionaryContents(
        directory="0", name="error"
    )["internalField"].val

    self.results = dict(avg_error=sum(error_field) / len(error_field),
                        max_error=max(error_field))
    

# Run the queue
while True:
    try:
        job_id = job_queue.get()

    except f3dasm.experiment.NoOpenJobsError:
        break

    args = data.get_inputdata_by_index(job_id)

    # Create simulation info
    material = openFoam.material.Material()
    geometry = openFoam.geometry.Geometry()
    boundary = openFoam.boundary.Boundary(
        parameters=dict(
            input_velocity=dict(
                value=args["input_velocity"],
                description="Velocity along the x direction at the inlet boundary.",
            ),
            output_pressure=dict(
                value=args["output_pressure"],
                description="Pressure along the x direction at the outlet boundary.",
            ),
        )
    )
    mesh = openFoam.mesh.Mesh(
        parameters=dict(
            mesh_factor=dict(
                value=args["mesh_factor"],
                description="Mesh size multiplier",
            ),
        )
    )
    solver = openFoam.solver.Solver()
    simulation_info = openFoam.simulator_info.SimulationInfo(
        material=material,
        geometry=geometry,
        boundary=boundary,
        mesh=mesh,
        solver=solver,
        case_source_path=Path("cylinder.orig"),
        job_id=job_id,
    )

    simulator_info = openFoam.simulator_info.SimulatorInfo(
        fork="",
        version="",
        build_date="",
        preprocessors=[
            dict(preprocessor="prepareCase", options=["--no-mesh-create"]),
            dict(preprocessor="blockMesh", options=[]),
        ],
        solvers=[
            dict(
                solver="auto", options=["-withFunctionObjects", "-writePhi", "-writep"]
            )
        ],
        postprocessors=[
            dict(
                postprocessor="postProcess",
                options=["-func", "streamFunction"],
                post_func=error_calculations,
            )
        ],
    )

    simulator = openFoam.openfoam_simulator.openFoamSimulator(
        simulation_info=simulation_info, simulator_info=simulator_info
    )

    simulator.pre_process(overwrite=True)

    simulator.execute()

    simulator.post_process()

    data.set_outputdata_by_index(job_id, itemgetter("avg_error", "max_error")(simulator.results))

    data.store("data")

    job_queue.mark_finished(job_id)


2023-06-27 00:17:42,188 - Starting job cylinder_0 at /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0
 PyFoam WARNING on line 698 of file /home/guillaume/miniconda3/envs/f3dasm_env/lib/python3.10/site-packages/PyFoam/RunDictionary/SolutionDirectory.py : The first timestep in /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0  is  None not a number. Doing nothing 
2023-06-27 00:17:42,314 - Starting: blockMesh -case /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0 in /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder
2023-06-27 00:17:42,330 - Started with PID 53357
2023-06-27 00:17:43,024 - Finished
2023-06-27 00:17:43,046 - Automatic mode will use potentialFoam solver.
2023-06-27 00:17:43,049 - Starting: potentialFoam -case /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0 -withFunctionObjects -writeP

## Output the results

In [5]:
data.data

input                                   output          
  mesh_factor input_velocity output_pressure avg_error max_error
0          10       3.807947        0.950714  0.053586  0.091391
1           3       7.346740        0.598658  0.053992  0.089828
2           7       1.644585        0.155995  0.053669  0.091134
3           4       0.675028        0.866176  0.053810  0.090438
4           9       6.051039        0.708073  0.053696  0.091337
5           3       0.303786        0.969910  0.053992  0.089828
6           5       8.341182        0.212339  0.053693  0.090716
7           3       1.900067        0.183405  0.053992  0.089828
8           7       3.111998        0.524756  0.053669  0.091134
9           5       4.376256        0.291229  0.053693  0.090716